In [1]:
import getpass
import os
from pathlib import Path
from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()

In [2]:
from agno.models.huggingface import HuggingFace
from agno.embedder.ollama import OllamaEmbedder
from agno.models.ollama import Ollama
from agno.models.huggingface import HuggingFace
from agno.models.groq import Groq
from agno.agent import Agent
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.csv_toolkit import CsvTools


* LLM

In [3]:
hf_model=HuggingFace(
        id="meta-llama/Meta-Llama-3-8B-Instruct",
        api_key=os.getenv("HUGGINGFACE_API_TOKEN"),
)


ollama_model=Ollama(id="llama3.2:3b")

groq_model=Groq(id="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API_KEY"))

In [4]:
ollama_embedder = OllamaEmbedder(id="nomic-embed-text:latest",dimensions=768)

### Web Searcher Agent

In [5]:
web_searcher = Agent(model=groq_model, 
                     tools=[DuckDuckGoTools()], 
                     name="Web Searcher",  
                     role="Searches the web for information on a topic", 
                     show_tool_calls=True,
                     markdown=False)

response = web_searcher.run("Who is Thomas Edison?")
print(response.content) #latencia 1min 50

 - Running: duckduckgo_search(query=Thomas Edison, max_results=5)

Based on the search results, it appears that Thomas Edison was a prolific inventor and businessman who is considered one of America's leading innovators. He developed many devices in fields such as electric power generation, mass communication, sound recording, and motion pictures, and is credited with inventing the phonograph, the motion picture camera, and early versions of the electric light bulb.


###  RAG Agent

In [6]:
from agno.vectordb.lancedb import LanceDb, SearchType

# Define the database URL where the vector database will be stored
db_url = "./tmp/lancedb"
vector_db = LanceDb(
    table_name="recipes",  # Table name in the vector database
    uri=db_url,  # Location to initiate/create the vector database
    embedder=ollama_embedder,  # Without using this, it will use OpenAIChat embeddings by default
    search_type=SearchType.vector,
)

In [7]:
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
# Create a knowledge base of PDFs from URLs
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=vector_db
)
# Load the knowledge base
knowledge_base.load(recreate=False)

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO     Added 0 documents to knowledge base

In [8]:
# Create and use the agent
rag_agent = Agent(model=groq_model,
              name="RAG Agent",
              role="Responds to questions about Thai recipes", 
              knowledge=knowledge_base,  

              # Add a tool to search the knowledge base which enables agentic RAG.
               # This is enabled by default when `knowledge` is provided to the Agent.
              search_knowledge=False,
              show_tool_calls=True,
              markdown=False)


In [12]:
rag_agent.run("How do I make chicken and galangal in coconut milk soup")

RunResponse(content="Tom Kha Gai is a popular Thai dish made with chicken, galangal, and coconut milk. Here's a simple recipe to make a delicious Tom Kha Gai:\n\n**Ingredients:**\n\n- 2 lbs boneless, skinless chicken breast or thighs, cut into bite-sized pieces\n- 2-3 inches of galangal (or 2-3 cloves of garlic), peeled and grated\n- 2 lemongrass stalks, bruised\n- 2-3 kaffir lime leaves\n- 2 cups of coconut milk\n- 2 cups of chicken broth\n- 1 tablespoon of fish sauce\n- 1 tablespoon of palm sugar (or brown sugar)\n- Salt and pepper, to taste\n- Fresh Thai basil leaves, for garnish\n- Chilies (optional), for serving\n\n**Instructions:**\n\n1. **Prepare the galangal and lemongrass**: In a blender or food processor, combine the galangal, lemongrass, and kaffir lime leaves. Blend until you get a smooth paste. You can also use a mortar and pestle to pound the ingredients until they're well combined.\n\n2. **Sear the chicken**: Heat 2 tablespoons of oil in a large pot over medium-high heat

### Multi-Agent Team

In [9]:
team_leader = Agent(
    name="Multi-Agent Team",
    role="Team leader agent",
    model=groq_model,
    tools=[web_searcher,rag_agent],
    show_tool_calls=True,
    markdown=False,
    debug_mode=False,
)


In [10]:
team_leader.run("Who is Thomas Edison?")

RunResponse(content='A legendary figure in the history of innovation and technology. Thomas Edison (1847-1931) was an American inventor, entrepreneur, and engineer who held over 1,093 patents in his name, making him one of the most prolific inventors in history.\n\nEdison is credited with numerous groundbreaking inventions and innovations that transformed the world. Some of his most notable achievements include:\n\n1. **Development of the Light Bulb**: Edison famously declared, "I have not failed. I\'ve just found 10,000 ways that won\'t work." His endless experimentation led to the creation of the first commercially viable incandescent light bulb, which paved the way for widespread use in homes, businesses, and industries.\n2. **Phonograph and Sound Recording**: Edison invented the phonograph, the first device capable of recording and playing back sound, in 1877. This invention revolutionized music, speech, and entertainment.\n3. **Motion Pictures**: Edison developed the kinetoscope, 

In [13]:
team_leader.run("How do I make chicken and galangal in coconut milk soup")

RunResponse(content="A classic Thai-inspired dish. Here's a recipe for Chicken and Galangal in Coconut Milk Soup, also known as Tom Kha Gai:\n\n**Ingredients:**\n\n* 1 kg boneless, skinless chicken breast or thighs, cut into bite-sized pieces\n* 2-3 inches galangal (or Thai ginger), peeled and chopped\n* 2 lemongrass stalks, bruised\n* 2 kaffir lime leaves\n* 2 cloves garlic, minced\n* 1 tablespoon grated fresh ginger\n* 1 tablespoon Thai red curry paste\n* 2 cups coconut milk\n* 2 cups chicken broth\n* 1 tablespoon palm sugar (or brown sugar)\n* 1 tablespoon fish sauce (optional)\n* Salt and pepper to taste\n* Fresh cilantro or basil leaves for garnish\n* Lime wedges for serving\n\n**Instructions:**\n\n1. **Prepare the ingredients:** Pound the galangal, lemongrass, and kaffir lime leaves together in a mortar and pestle or use a rolling pin to release their flavors and oils.\n2. **Heat the oil:** In a large pot or wok, heat 2 tablespoons of vegetable oil over medium heat.\n3. **Add aro